# Simularium Conversion Tutorial : NERDSS PDB Data

In [1]:
from IPython.display import Image

import numpy as np

import os
import random

from simulariumio.nerdss import NerdssConverter, NerdssData
from simulariumio import MetaData, DisplayData, DISPLAY_TYPE, CameraData, UnitData
from simulariumio.filters import TranslateFilter
from simulariumio.writers import JsonWriter
from simulariumio.writers import BinaryWriter



This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

# _Note:_
To install simulariumio with all depencies needed for PDB, use `pip install simulariumio`

***
## Prepare your spatial data

The Simularium `NerdssConverter` consumes spatiotemporal data from NERDSS outputs using the MDAnalysis Python package. 

The converter requires a `NerdssData` object as a parameter ([see documentation](https://simularium.github.io/simulariumio/simulariumio.nerdss.html#simulariumio.nerdss.nerdss_data.NerdssData))

If you'd like to specify radii or color for rendering an agent type, add a `DisplayData` object for that agent type, as shown below ([see documentation](https://simularium.github.io/simulariumio/simulariumio.data_objects.html#module-simulariumio.data_objects.display_data)).


In [3]:
# Define the folder containing the PDB files
folder_path = "../simulariumio/tests/data/nerdss/clathrin"

# List all files in the directory
all_files = os.listdir(folder_path)

# Filter to include only .pdb files
pdb_files = [file for file in all_files if file.endswith('.pdb')]

# Check if there are any PDB files in the folder
if not pdb_files:
    print("No PDB files found in the folder.")
else:
    # Choose one PDB file at random
    chosen_file = random.choice(pdb_files)
    chosen_file_path = os.path.join(folder_path, chosen_file)
    
    # Read the first few lines of the chosen PDB file
    with open(chosen_file_path, 'r') as file:
        first_lines = [next(file).strip() for _ in range(5)]  # Adjust the range to read more or fewer lines
        second_line = first_lines[1].strip()  # Read the second line
    
    parts = second_line.split()
    numbers = [float(parts[1]), float(parts[2]), float(parts[3])]
    
    # Convert the numbers into a numpy array
    box_array = np.array(numbers)
    
    #print(box_array)


In [4]:
example_data = NerdssData(
    path_to_pdb_files="../simulariumio/tests/data/nerdss/clathrin",
    meta_data=MetaData(
        box_size=box_array,
        trajectory_title="Some parameter set",
        camera_defaults=CameraData(position=np.array([0, 0, 200]))
    ),
#     display_data={
#         "gag#COM": DisplayData(
#             name="GAG - Center of Mass",
#             display_type=DISPLAY_TYPE.SPHERE,
#             color="#0000FF",
#         ),
#         "pol#COM": DisplayData(
#             name="POL - Center of Mass",
#             display_type=DISPLAY_TYPE.SPHERE,
#             color="#FF00FF",
#         ),
#     },
     time_units=UnitData("us", 3), #declare simulation time step in microseconds
)

## Convert and save as .simularium file

Once your data is shaped like in the `example_data` object, you can use the converter to generate the file at the given path:

(since this model's coordinates are all positive, use a `TranslateFilter` to center the data in the viewer.)

In [5]:
converter = NerdssConverter(example_data)
# this _filter is just roughly centering the data in the box
_filter = TranslateFilter(default_translation=box_array/-2)
filtered_data = converter.filter_data([_filter])
#JsonWriter.save(filtered_data, "example_virus", False)
BinaryWriter.save(filtered_data, "example_clathrin", False)

Reading PDB Data -------------


C:\Users\hassa\anaconda3\lib\site-packages\MDAnalysis\topology\PDBParser.py:331: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
C:\Users\hassa\anaconda3\lib\site-packages\MDAnalysis\topology\guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: M
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
C:\Users\hassa\anaconda3\lib\site-packages\MDAnalysis\topology\guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: R
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
C:\Users\hassa\anaconda3\lib\site-packages\MDAnalysis\topology\guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: T
  warnings.warn("Failed to guess the mass for the follo

Filtering: translation -------------
Converting Trajectory Data to Binary -------------
Writing Binary -------------
saved to example_clathrin.simularium


## Visualize in the Simularium viewer

In a supported web-browser (Firefox or Chrome), navigate to https://simularium.allencell.org/ and import your file into the view.